In [3]:
import pandas as pd

df = pd.read_csv('http://web.mta.info/developers/data/nyct/turnstile/turnstile_190803.txt')
    
    # rename columns to get rid of the funny 'EXITS     ' issue
df.columns = ['C/A','UNIT','SCP','STATION','LINENAME',\
                  'DIVISION','DATE','TIME','DESC','ENTRIES','EXITS']
    
    # convert to datetime/make turnstile column
df['converted_time'] = pd.to_datetime(df['DATE']+' '+df['TIME'])
df['turnstiles'] = df['C/A'] + '-' + df['UNIT'] + '-' + df['SCP'] + '-' + df['STATION']
    
    # sort by date and location
df_sorted = df.sort_values(['turnstiles', 'converted_time'])
    
    # group by turnstile so we can get entry/exit differences
turnstile_df = df_sorted.groupby('turnstiles')
df_sorted['entries_diff'] = turnstile_df['ENTRIES'].diff()
df_sorted['exits_diff'] = turnstile_df['EXITS'].diff()
    
    # calculates IQR for entries_diff
Q3 = df_sorted['entries_diff'].quantile(0.75) 
Q1 = df_sorted['entries_diff'].quantile(0.25)
IQR = Q3 - Q1
    
    # calculates IQR range using outliers 
IQR_range = (Q1 - 1.5 * IQR, Q3 + 1.5 * IQR)
    
    # removes values outside of lower and upper bounds
df_sorted = df_sorted[df_sorted['entries_diff'].between(0, IQR_range[1])]
    
    # repeats process for exits_diff
Q3_2 = df_sorted['exits_diff'].quantile(0.75) 
Q1_2 = df_sorted['exits_diff'].quantile(0.25)
IQR_2 = Q3_2 - Q1_2
IQR_range_2 = (Q1_2 - 1.5 * IQR, Q3_2 + 1.5 * IQR)
df_sorted = df_sorted[df_sorted['exits_diff'].between(0, IQR_range_2[1])]

    # replaces NaN values with mean for entries_diff and exits_diff
df_sorted.entries_diff = df_sorted.entries_diff.fillna(df_sorted.entries_diff.mean())
df_sorted.exits_diff = df_sorted.exits_diff.fillna(df_sorted.exits_diff.mean())
    
print(df_sorted.info())

<class 'pandas.core.frame.DataFrame'>
Int64Index: 175365 entries, 1 to 206995
Data columns (total 15 columns):
 #   Column          Non-Null Count   Dtype         
---  ------          --------------   -----         
 0   C/A             175365 non-null  object        
 1   UNIT            175365 non-null  object        
 2   SCP             175365 non-null  object        
 3   STATION         175365 non-null  object        
 4   LINENAME        175365 non-null  object        
 5   DIVISION        175365 non-null  object        
 6   DATE            175365 non-null  object        
 7   TIME            175365 non-null  object        
 8   DESC            175365 non-null  object        
 9   ENTRIES         175365 non-null  int64         
 10  EXITS           175365 non-null  int64         
 11  converted_time  175365 non-null  datetime64[ns]
 12  turnstiles      175365 non-null  object        
 13  entries_diff    175365 non-null  float64       
 14  exits_diff      175365 non-null  flo